In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

## goal of this homework is to create a regression model for predicting housing prices (column 'median_house_value').
# EDA
- Load the data.
- Look at the median_house_value variable. Does it have a long tail?

# Features
## For the rest of the homework, you'll need to use only these columns:
- 'latitude',
- 'longitude',
- 'housing_median_age',
- 'total_rooms',
- 'total_bedrooms',
- 'population',
- 'households',
- 'median_income',
- 'median_house_value'
## Select only them.

In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv')

In [24]:
select_columns = ['latitude', 'longitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population',
                  'households', 'median_income', 'median_house_value']
df = df[select_columns]
df.head(2)

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,37.88,-122.23,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,37.86,-122.22,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0


# Question 1
## Find a feature with missing values. How many missing values does it have?
- 207
- 307
- 408
- 508

In [27]:
df.isna().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

# Question 2
## What's the median (50% percentile) for variable 'population'?

- 1133
- 1122
- 1166
- 1188



In [28]:
df.population.median()

1166.0

Split the data
Shuffle the initial dataset, use seed 42.
Split your data in train/val/test sets, with 60%/20%/20% distribution.
Make sure that the target value ('median_house_value') is not in your dataframe.
Apply the log transformation to the median_house_value variable using the np.log1p() function.

In [29]:
n = len(df)
train = n

20640